In [ ]:
import json
import os
import wave
import numpy as np
import pandas as pd
import librosa
from pydub import AudioSegment
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from vosk import Model, KaldiRecognizer
import jiwer

# Константы
DATA_DIR = '../data/train/'
DATA_DIR_FILES = [
    'hr_bot_clear',
    'hr_bot_noise',
    'hr_bot_synt'
]
ANNOTATION_DIR = '../data/train/annotation/'
ANNOTATION_FILES = [
    'hr_bot_clear.json',
    'hr_bot_noise.json',
    'hr_bot_synt.json'
]

VAL_DIR = '../data/val/luga/'  # Путь к валидационным данным
ANNOTATION_VAL_FILE = os.path.join(VAL_DIR, 'luga.json')

# Настройка Vosk модели для распознавания речи
MODEL_PATH = "../model/vosk_model"  # Путь к скачанной модели Vosk
vosk_model = Model(MODEL_PATH)

# Нейронная сеть для классификации текста
class TextClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(TextClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Датасет для классификации текста
class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

# Преобразование MP3 в WAV
def convert_mp3_to_wav(mp3_filepath):
    try:
        wav_filepath = mp3_filepath.replace('.mp3', '.wav')
        audio = AudioSegment.from_mp3(mp3_filepath)
        audio.export(wav_filepath, format='wav')
        return wav_filepath
    except Exception as e:
        print(f"Ошибка при конвертации {mp3_filepath} в WAV: {e}")
        return None

# Функция загрузки аннотаций
def load_annotations(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

# Загрузка датасета с аннотациями
def load_dataset(data_dir, annotation_files):
    dataset = []
    for annotation_file in annotation_files:
        annotations = load_annotations(os.path.join(ANNOTATION_DIR, annotation_file))
        sub_dir = annotation_file.replace('.json', '')
        for item in annotations:
            if all(k in item for k in ['audio_filepath', 'text', 'label', 'attribute']):
                audio_path = os.path.join(data_dir, sub_dir, item['audio_filepath'])
                if os.path.exists(audio_path):
                    if audio_path.endswith('.mp3'):
                        audio_path = convert_mp3_to_wav(audio_path)
                    if audio_path:
                        dataset.append({
                            'audio_filepath': audio_path,
                            'text': item['text'],
                            'label': item['label'],
                            'attribute': item['attribute']
                        })
    return dataset

# Загрузка данных
dataset = load_dataset(DATA_DIR, ANNOTATION_FILES)
df = pd.DataFrame(dataset)

# Функция извлечения MFCC признаков
def extract_features(audio_filepath):
    try:
        signal, sr = librosa.load(audio_filepath, sr=None)
        mfccs = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=13)
        return np.mean(mfccs.T, axis=0)
    except Exception as e:
        print(f"Ошибка при обработке {audio_filepath}: {e}")
        return None

# Извлечение признаков
df['features'] = df['audio_filepath'].apply(extract_features)
df = df.dropna()  # Удаляем строки с ошибками

X = np.array(df['features'].tolist())
y = np.array(df['label'])

# Разделение данных на обучающую и тестовую выборки
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Определение модели
model = tf.keras.Sequential([
    layers.Input(shape=(13,)),  # Входной слой (13 признаков из MFCC)
    layers.Dense(64, activation='relu'),  # Первый скрытый слой
    layers.Dense(64, activation='relu'),  # Второй скрытый слой
    layers.Dense(len(set(y)), activation='softmax')  # Выходной слой (количество классов)
])

# Компиляция модели
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Обучение модели
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

# Оценка модели на тестовой выборке
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

# Сохранение модели
model.save('train_command_model.h5')

# Функция для распознавания речи с использованием Vosk
def transcribe_audio(audio_file):
    wf = wave.open(audio_file, "rb")
    rec = KaldiRecognizer(vosk_model, wf.getframerate())

    result_text = ""
    while True:
        data = wf.readframes(4000)
        if len(data) == 0:
            break
        if rec.AcceptWaveform(data):
            result = json.loads(rec.Result())
            result_text += result.get("text", "")

    final_result = json.loads(rec.FinalResult())
    result_text += final_result.get("text", "")

    return result_text

# Классы команд
commands = {
    0: "отказ",
    1: "отмена",
    2: "подтверждение",
    3: "начать осаживание",
    4: "осадить на (количество) вагон",
    5: "продолжаем осаживание",
    6: "зарядка тормозной магистрали",
    7: "вышел из межвагонного пространства",
    8: "продолжаем роспуск",
    9: "растянуть автосцепки",
    10: "протянуть на (количество) вагон",
    11: "отцепка",
    12: "назад на башмак",
    13: "захожу в межвагонное пространство",
    14: "остановка",
    15: "вперед на башмак",
    16: "сжать автосцепки",
    17: "назад с башмака",
    18: "тише",
    19: "вперед с башмака",
    20: "прекратить зарядку тормозной магистрали",
    21: "тормозить",
    22: "отпустить",
}

# Функция для классификации команды
def classify_command(text):
    for command in commands.values():
        if command in text:
            return command
    return "Unknown Command"

# Функция для оценки команд
def evaluate_commands():
    annotations = load_annotations(ANNOTATION_VAL_FILE)
    correct_predictions = 0
    total_predictions = len(annotations)

    for item in annotations:
        audio_filepath = item['audio_filepath']
        label = item['label']

        audio_path = os.path.join(VAL_DIR, audio_filepath)
        if os.path.exists(audio_path):
            transcription = transcribe_audio(audio_path)
            prediction = classify_command(transcription)
            print(f'Predicted: {prediction}, Actual: {label} for {audio_path}')

            if prediction == label:
                correct_predictions += 1

    accuracy = correct_predictions / total_predictions
    print(f'\nTotal Predictions: {total_predictions}, Correct Predictions: {correct_predictions}, Accuracy: {accuracy:.2f}')

evaluate_commands()
